# This script defines a complete workflow for generating a personalized skincare routine based on the user's input details.

# Necessary components for interfacing with Azure's OpenAI service, sets up a ChromaDB vector database for storing and retrieving text, defines a prompt template for generating skincare routines, and executes a sequential chain to process the user's input and generate a skincare recommendation.

In [1]:
pip install langchain-openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_openai import OpenAI

In [3]:
pip show langchain_openai

Name: langchain-openai
Version: 0.0.5
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /home/jupyter-seema/.local/lib/python3.10/site-packages
Requires: langchain-core, numpy, openai, tiktoken
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import json
from langchain_openai import AzureOpenAI, AzureOpenAIEmbeddings
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import Chroma
import chromadb

# Environment configuration
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["OPENAI_API_KEY"] = "549d3edec7cb431dbdcf18a6abdbb1a2"  # Use your actual OpenAI API key
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://seema.openai.azure.com/"  # Use your actual Azure OpenAI endpoint

# Embedding and database setup
embeddings = AzureOpenAIEmbeddings(azure_deployment="seema-embedding")
client = chromadb.HttpClient(host="20.118.32.222", port=8000)  # Use your actual ChromaDB host and port
vectordb = Chroma(embedding_function=embeddings, collection_name="Amanda", client=client)

# Initialize with your initial texts if needed
initial_texts = ["I have sensitive skin", "I like L'Oreal products", "Never recommend me sunscreen"]
vectordb.add_texts(initial_texts)

# Skincare routine prompt template
prompt_template = """
Hi, I am your personal AI assistant from AINA! How may I assist you today? This should output a skincare routine for the user.

A skincare routine has the following 7 steps:

1. Cleanser
2. Exfoliator
3. Toner
4. Serum
5. Eye cream
6. Moisturizer
7. Sunscreen

Recommend actual skincare products based on the user's preferences following these 7 steps, considering their name, age, gender, and skin tone.

Always output in JSON in this format: {{"step #": "product name"}}

Context: Name: {name}, Age: {age}, Gender: {gender}, Skin Tone: {skin_tone}

Question: What is the best skincare routine for {name}, a {age}-year-old {gender} with {skin_tone} skin?

Please answer just like a human, and ask any additional queries to fine-tune your response for a more personalized answer.

"""

# Creating the prompt template
skincare_routine_prompt = PromptTemplate(
    input_variables=['name', 'age', 'gender', 'skin_tone'],
    template=prompt_template
)

# Setting up memory and LLM
skincare_memory = ConversationBufferMemory(input_key='skin_tone', memory_key='routine_history')
llm = AzureOpenAI(deployment_name="seema-deployment")  # Use your actual deployment name

# Setting up the chain
skincare_chain = LLMChain(llm=llm, prompt=skincare_routine_prompt, verbose=True, output_key='skincare_routine', memory=skincare_memory)

parent_chain = SequentialChain(
    chains=[skincare_chain],
    input_variables=['name', 'age', 'gender', 'skin_tone'],
    output_variables=['skincare_routine'],
    verbose=True
)

def get_skincare_routine():
    print("Starting the skincare routine process...")  # Debug print

    input_details = {
        "name": input("Enter your name: ").strip(),
        "age": input("Enter your age: ").strip(),
        "gender": input("Enter your gender: ").strip(),
        "skin_tone": input("Enter your skin tone type: ").strip()
    }

    # Initialize skincare_routine_dict as an empty dictionary to ensure scope availability
    skincare_routine_dict = {}

    if all(input_details.values()):
        print("Input details collected:", input_details)  # Debug print

        try:
            print("Running the skincare chain...")  # Debug print
            result = parent_chain.run(input_details)
            print("Chain executed. Result:", result)  # Debug print to check if the chain returns a result

            # Process the result
            skincare_routine_dict = json.loads(result['skincare_routine'])  # Adjust according to actual result format

            print(f"\nSkincare Routine for {input_details['name']}:\n")
            print("Based on your age, gender, and skin type, here is a recommended skincare routine:\n")

            for step, product in skincare_routine_dict.items():
                print(f"{step}. {product}")
            
            print("\nPlease let me know if you have any specific preferences for certain ingredients or brands, and I will be happy to provide more personalized recommendations.")

        except Exception as e:
            print(f"An error occurred: {e}")  # This should catch and display any error

        # Check if skincare_routine_dict is not empty before attempting to add it to vectordb
        if skincare_routine_dict:
            vectordb.add_texts([json.dumps(skincare_routine_dict)])  # Only add if skincare_routine_dict has been successfully created
    else:
        print("Please ensure all details (name, age, gender, skin tone) are entered.")

if __name__ == "__main__":
    get_skincare_routine()


Starting the skincare routine process...


Enter your name:  Amanda
Enter your age:  34
Enter your gender:  Female
Enter your skin tone type:  oily


Input details collected: {'name': 'Amanda', 'age': '34', 'gender': 'Female', 'skin_tone': 'oily'}
Running the skincare chain...


> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Hi, I am your personal AI assistant from AINA! How may I assist you today? This should output a skincare routine for the user.

A skincare routine has the following 7 steps:

1. Cleanser
2. Exfoliator
3. Toner
4. Serum
5. Eye cream
6. Moisturizer
7. Sunscreen

Recommend actual skincare products based on the user's preferences following these 7 steps, considering their name, age, gender, and skin tone.

Always output in JSON in this format: {"step #": "product name"}

Context: Name: Amanda, Age: 34, Gender: Female, Skin Tone: oily

Question: What is the best skincare routine for Amanda, a 34-year-old Female with oily skin?

Please answer just like a human, and ask any additional queries to fine-tune your response for a more personalized answer.



> Finished c